In [2]:
from experta import *

class States(Fact):
    nm_State = Field(str, mandatory=True)
    nm_Biome = Field(str, mandatory=True)
    nm_Climate = Field(str, mandatory=True)
    sn_Coast = Field(str, mandatory=True)
    sn_Border = Field(str, mandatory=True)
    vl_Population = Field(float, mandatory=True)
    vl_COVID = Field(float, mandatory=True)
    vl_PIB = Field(float, mandatory=True)

class Classificator(Fact):

    @DefFacts()
    def Init_Facts():
        yield States(nm_State='Acre', nm_Biome='Amazonia' ,nm_Climate='Equatorial' , sn_Coast='N', sn_Border='S' , vl_Population=906876 , vl_COVID=1816, vl_PIB=15331)
        yield States(nm_State='Alagoas', nm_Biome='Caatinga' ,nm_Climate='Semiárido' , sn_Coast='S', sn_Border='N' , vl_Population=3365351 , vl_COVID=6141, vl_PIB=54413)
        yield States(nm_State='Amapá', nm_Biome='Amazonia' ,nm_Climate='Equatorial' , sn_Coast='S', sn_Border='S' , vl_Population=877613 , vl_COVID=1962, vl_PIB=16795)
        yield States(nm_State='Amazonas', nm_Biome='Amazonia' ,nm_Climate='Equatorial' , sn_Coast='N', sn_Border='S' , vl_Population=4269995 , vl_COVID=13705, vl_PIB=100109)
        yield States(nm_State='Bahia', nm_Biome='Mata Atlântica' ,nm_Climate='Tropical' , sn_Coast='S', sn_Border='N' , vl_Population=14985284 , vl_COVID=26650, vl_PIB=286240)
        yield States(nm_State='Ceará', nm_Biome='Caatinga' ,nm_Climate='Semiárido' , sn_Coast='S', sn_Border='N' , vl_Population=9240580 , vl_COVID=24127, vl_PIB=155904)
        yield States(nm_State='Distrito Federal', nm_Biome='Cerrado' ,nm_Climate='Tropical' , sn_Coast='N', sn_Border='N' , vl_Population=3094325 , vl_COVID=10222, vl_PIB=254817)
        yield States(nm_State='Espiríto Santo', nm_Biome='Mata Atlântica' ,nm_Climate='Tropical' , sn_Coast='S', sn_Border='N' , vl_Population=4108508 , vl_COVID=12369, vl_PIB=137020)
        yield States(nm_State='Goiás', nm_Biome='Cerrado' ,nm_Climate='Tropical' , sn_Coast='N', sn_Border='N' , vl_Population=7206589 , vl_COVID=22974, vl_PIB=195682)
        yield States(nm_State='Maranhão', nm_Biome='Amazonia' ,nm_Climate='Equatorial' , sn_Coast='S', sn_Border='N' , vl_Population=7153262 , vl_COVID=10106, vl_PIB=98179)
        yield States(nm_State='Mato Grosso', nm_Biome='Pantanal' ,nm_Climate='Equatorial' , sn_Coast='N', sn_Border='S' , vl_Population=3567234 , vl_COVID=13655, vl_PIB=137443)
        yield States(nm_State='Mato Grosso do Sul', nm_Biome='Pantanal' ,nm_Climate='Tropical' , sn_Coast='N', sn_Border='S' , vl_Population=2839188 , vl_COVID=9468, vl_PIB=106969)
        yield States(nm_State='Minas Gerais', nm_Biome='Cerrado' ,nm_Climate='Tropical' , sn_Coast='N', sn_Border='N' , vl_Population=21411923 , vl_COVID=53698, vl_PIB=614876)
        yield States(nm_State='Pará', nm_Biome='Amazonia' ,nm_Climate='Equatorial' , sn_Coast='S', sn_Border='S' , vl_Population=8777124 , vl_COVID=38116, vl_PIB=440029)
        yield States(nm_State='Paraíba', nm_Biome='Caatinga' ,nm_Climate='Semiárido' , sn_Coast='S', sn_Border='N' , vl_Population=4059905 , vl_COVID=9246, vl_PIB=64374)
        yield States(nm_State='Paraná', nm_Biome='Mata Atlântica' ,nm_Climate='Subtropical' , sn_Coast='S', sn_Border='S' , vl_Population=11597484 , vl_COVID=16544, vl_PIB=161350)
        yield States(nm_State='Pernambuco', nm_Biome='Caatinga' ,nm_Climate='Semiárido' , sn_Coast='S', sn_Border='N' , vl_Population=9674793 , vl_COVID=19552, vl_PIB=186352)
        yield States(nm_State='Piauí', nm_Biome='Caatinga' ,nm_Climate='Semiárido' , sn_Coast='S', sn_Border='N' , vl_Population=3289290 , vl_COVID=6980, vl_PIB=50378)
        yield States(nm_State='Rio de Janeiro', nm_Biome='Mata Atlântica' ,nm_Climate='Tropical' , sn_Coast='S', sn_Border='N' , vl_Population=17463349 , vl_COVID=63880, vl_PIB=758859)
        yield States(nm_State='Rio Grande do Norte', nm_Biome='Caatinga' ,nm_Climate='Semiárido' , sn_Coast='S', sn_Border='N' , vl_Population=3560903 , vl_COVID=7302, vl_PIB=66970)
        yield States(nm_State='Rio Grande do Sul', nm_Biome='Pampa' ,nm_Climate='Subtropical' , sn_Coast='S', sn_Border='S' , vl_Population=11466630 , vl_COVID=34462, vl_PIB=457294)
        yield States(nm_State='Rondônia', nm_Biome='Amazonia' ,nm_Climate='Equatorial' , sn_Coast='N', sn_Border='S' , vl_Population=1815278 , vl_COVID=6506, vl_PIB=44914)
        yield States(nm_State='Roraima', nm_Biome='Amazonia' ,nm_Climate='Equatorial' , sn_Coast='N', sn_Border='S' , vl_Population=652713 , vl_COVID=1968, vl_PIB=13370)
        yield States(nm_State='Santa Catarina', nm_Biome='Mata Atlântica' ,nm_Climate='Subtropical' , sn_Coast='S', sn_Border='S' , vl_Population=7338473 , vl_COVID=18953, vl_PIB=298227)
        yield States(nm_State='São Paulo', nm_Biome='Mata Atlântica' ,nm_Climate='Tropical' , sn_Coast='S', sn_Border='N' , vl_Population=46649132 , vl_COVID=147258, vl_PIB=2210562)
        yield States(nm_State='Sergipe', nm_Biome='Caatinga' ,nm_Climate='Semiárido' , sn_Coast='S', sn_Border='N' , vl_Population=2338474 , vl_COVID=6003, vl_PIB=42018)
        yield States(nm_State='Tocantins', nm_Biome='Cerrado' ,nm_Climate='Tropical' , sn_Coast='N', sn_Border='N' , vl_Population=1607363 , vl_COVID=3716, vl_PIB=35666)

    @Rule()
    def Ask_Vegetation():
        self.declare(Fact())


engine = States()
engine.reset()
engine.run()

AttributeError: 'States' object has no attribute 'reset'